In [ ]:
pip install --upgrade keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.4.3)


# Clonamos el repositorio para obtener el dataset

In [1]:
!git clone https://github.com/joanby/deeplearning-az.git

Cloning into 'deeplearning-az'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 10135 (delta 14), reused 26 (delta 10), pack-reused 10096
Receiving objects: 100% (10135/10135), 236.92 MiB | 35.42 MiB/s, done.
Resolving deltas: 100% (39/39), done.
Checking out files: 100% (10106/10106), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Parte 1 - Construir el modelo de CNN

# Importar las librerías y paquetes

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Inicializar la CNN

In [2]:
classifier = Sequential()


# Paso 1 - Convolución

In [3]:
classifier.add(Conv2D(filters = 32,kernel_size = (3, 3), 
                      input_shape = (64, 64, 3), activation = "relu"))


# Paso 2 - Max Pooling

In [4]:
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Una segunda capa de convolución y max pooling

In [5]:
classifier.add(Conv2D(filters = 32,kernel_size = (3, 3), activation = "relu"))

In [6]:
classifier.add(MaxPooling2D(pool_size = (2,2)))


# Paso 3 - Flattening

In [7]:
classifier.add(Flatten())

# Paso 4 - Full Connection

In [8]:
classifier.add(Dense(units = 128, activation = "relu"))
classifier.add(Dense(units = 1, activation = "sigmoid"))

# Compilar la CNN
# Como va a ser entrenada?

In [9]:
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])


# Parte 2 - Ajustar la CNN a las imágenes para entrenar 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_dataset = train_datagen.flow_from_directory('dataset/training_set',
                                                    target_size=(64, 64),
                                                    batch_size=32,
                                                    class_mode='binary')

testing_dataset = test_datagen.flow_from_directory('dataset/test_set',
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='binary')

classifier.fit_generator(training_dataset,
                        steps_per_epoch=10,
                        epochs=25,
                        validation_data=testing_dataset,
                        validation_steps=2000)



Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25
10/10 [==============================] - 312s 31s/step - loss: 0.6936 - accuracy: 0.4656 - val_loss: 0.6924 - val_accuracy: 0.5725
Epoch 2/25
10/10 [==============================] - ETA: 0s - loss: 0.6902 - accuracy: 0.5250

# Parte 3 - Cómo hacer nuevas predicciones

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/deeplearning-az/datasets/Part 2 - Convolutional Neural Networks (CNN)/dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_dataset.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

